# Database initialization

In [ ]:
# todo: create a script to build the logical model in the chosen NOSQL system

In [9]:
# Connection to Neo4j
from neo4j import GraphDatabase

URI = "bolt://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "neo4jproject123"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD), database="test")

In [10]:
def run_query(query, params=None):
    with driver.session() as session:
        return session.run(query, params or {})

In [20]:
query = """
MATCH (n) DETACH DELETE n
"""
run_query(query)
print("Database cleared.")


Database cleared.


In [21]:
#Costrains
constraints = [
    "CREATE CONSTRAINT customer_id IF NOT EXISTS FOR (c:Customer) REQUIRE c.id IS UNIQUE",
    "CREATE CONSTRAINT terminal_id IF NOT EXISTS FOR (t:Terminal) REQUIRE t.id IS UNIQUE",
    "CREATE CONSTRAINT transaction_id IF NOT EXISTS FOR (tr:Transaction) REQUIRE tr.id IS UNIQUE"
]

for c in constraints:
    run_query(c)

print("Constraints created.")


Constraints created.


In [17]:
#Nodes
def create_customer(customer):
    query = """
    MERGE (c:Customer {id: $id})
    SET c.lat = $lat,
        c.lon = $lon,
        c.amountMean = $amountMean,
        c.amountStd = $amountStd,
        c.meanDailyTransactions = $meanDailyTransactions
    """
    run_query(query, customer)

def create_terminal(terminal):
    query = """
    MERGE (t:Terminal {id: $id})
    SET t.lat = $lat,
        t.lon = $lon
    """
    run_query(query, terminal)

def create_transaction(transaction):
    query = """
    MERGE (tr:Transaction {id: $id})
    SET tr.dateTime = datetime($dateTime),
        tr.amount = $amount,
        tr.isFraud = $isFraud
    """
    run_query(query, transaction)


In [18]:
#Relationships
def link_customer_terminal(customer_id, terminal_id):
    query = """
    MATCH (c:Customer {id: $customer_id})
    MATCH (t:Terminal {id: $terminal_id})
    MERGE (c)-[:USE]->(t)
    """
    run_query(query, {"customer_id": customer_id, "terminal_id": terminal_id})

def link_customer_transaction(customer_id, transaction_id):
    query = """
    MATCH (c:Customer {id: $customer_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (c)-[:PERFORM]->(tr)
    """
    run_query(query, {"customer_id": customer_id, "transaction_id": transaction_id})

def link_terminal_transaction(terminal_id, transaction_id):
    query = """
    MATCH (t:Terminal {id: $terminal_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (t)-[:ON]->(tr)
    """
    run_query(query, {"terminal_id": terminal_id, "transaction_id": transaction_id})


In [ ]:
driver.close()
print("Connection closed.")
